# Project idea

This project quantifies and analyzes changes in CORINE Land Cover and ecosystem services within a 5 km buffer around Central Macedonia's main roads between 2006 and 2012. It integrates spatial analysis with an interactive Streamlit dashboard to reveal the ecological impacts of transportation infrastructure over time.

## Data

In [13]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
import osmnx
import matplotlib

Set the data and output directories

In [9]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"
OUTPUT_DIRECTORY = NOTEBOOK_PATH / "docs"

Function for downloading data from the web as zip and extract them on the data directory.

In [18]:
import requests
import zipfile
import io

# Function to download and extract a zip file
def download_and_extract_zip(zip_url, extract_to="data"):
    response = requests.get(zip_url)
    response.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(extract_to)

### Corine Land Cover

Corine Land Cover data was downloaded from [data.ktimatologio.gr](https://data.ktimatologio.gr/)

In [ ]:
# Urls for the Corine Land Cover data
zip_url1 = "https://data.ktimatologio.gr/sites/default/files/2021-10/CLC06_GR.zip"
download_and_extract_zip(zip_url1)

zip_url2 = "https://data.ktimatologio.gr/sites/default/files/2021-10/CLC12_GR.zip"
download_and_extract_zip(zip_url2)

# Convert to GeoDataFrames
corine06 = gpd.read_file(
    DATA_DIRECTORY
    /"CLC06_GR"
    /"CLC06_GR.shp")   

corine12 = gpd.read_file(
    DATA_DIRECTORY
    /"CLC12_GR"
    /"CLC12_GR.shp")

### Ecosystem services

Data...

In [ ]:
# Import the data
eco_services = pd.read_csv(
    DATA_DIRECTORY
    / "eco_services.csv",
    sep=";",
    header=0,
)

# Select columns of interest
eco_services = eco_services[["Code",
                            "Ecological integrity (steiles B - H)",
                             'Provisioning ecosystem services (J - T)',
                             'Regulating ecosystem services (V - AD)',
                             'Cultural ecosystem services (AF - AG)'
                             ]]

# Rename columns
new_names = {
    "Ecological integrity (steiles B - H)": "ecological_integrity",
    "Provisioning ecosystem services (J - T)": "provisioning_services",
    "Regulating ecosystem services (V - AD)": "regulating_services",
    "Cultural ecosystem services (AF - AG)": "cultural_services",
}
eco_services = eco_services.rename(columns=new_names)

### Municipality Data

Municipality data was downloaded from [geodata.gov.gr](http://geodata.gov.gr/)

In [ ]:
# Import the data
zip_url3 = "http://geodata.gov.gr/dataset/63786e9f-7be9-4d1e-99c9-48ff45d0962f/resource/6643d54a-f1af-4841-ad99-7a49a1d13650/download/oriadhmwnkallikraths.zip"
download_and_extract_zip(zip_url3)

# Convert to GeoDataFrame
grid = gpd.read_file(
    DATA_DIRECTORY
    / "oria_dhmwn_kallikraths"
    / "oria_dhmwn_kallikraths.shp",
    encoding="ISO-8859-7" # for greek letters
)

### OSM Data

In [43]:
central_mac = "Περιφέρεια Κεντρικής Μακεδονίας"

graph_central_mac = osmnx.graph_from_place(central_mac, network_type="drive")
edges = osmnx.graph_to_gdfs(graph_central_mac)
area_mac = osmnx.geocode_to_gdf(central_mac)

area_mac = area_mac.to_crs(2100)
edges = edges.to_crs(2100)
graph_central_mac = osmnx.project_graph(graph_central_mac, to_crs=2100)

c:\Users\thano\anaconda3\envs\autogis\lib\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 11 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [86]:
edges.head()

osmid      highway  oneway reversed  \
u         v           key                                           
135401287 8274769613  0    27013165     tertiary   False    False   
          8274640348  0    27213549    secondary   False    False   
          8274769525  0    27222414    secondary   False     True   
135827942 11244868511 0    14203818  residential    True    False   
          1775534231  0    27031273     tertiary   False     True   

                                length  \
u         v           key                
135401287 8274769613  0     780.385241   
          8274640348  0      58.962948   
          8274769525  0    1179.400693   
135827942 11244868511 0      71.498639   
          1775534231  0     159.368033   

                                                                    geometry  \
u         v           key                                                      
135401287 8274769613  0    LINESTRING (493574.364 4423509.696, 493560.086...   
          8274640348  0    LINESTRING (493574.364 4423509.696, 493577.942...   
          8274769525  0    LINESTRING (493574.364 4423509.696, 493570.752...   
135827942 11244868511 0    LINESTRING (484122.386 4448029.916, 484129.954...   
          1775534231  0    LINESTRING (484122.386 4448029.916, 484126.628...   

                          lanes                       name maxspeed width  \
u         v           key                                                   
135401287 8274769613  0     NaN                        NaN      NaN   NaN   
          8274640348  0       2  Περιμετρική Οδός Σιθωνίας      NaN   NaN   
          8274769525  0       2  Περιμετρική Οδός Σιθωνίας      NaN   NaN   
135827942 11244868511 0       1                        NaN       30     5   
          1775534231  0       2                        NaN       30     5   

                          access bridge  ref junction tunnel est_width  \
u         v           key                                                
135401287 8274769613  0      NaN    NaN  NaN      NaN    NaN       NaN   
          8274640348  0      NaN    NaN  NaN      NaN    NaN       NaN   
          8274769525  0      NaN    NaN  NaN      NaN    NaN       NaN   
135827942 11244868511 0      NaN    NaN  NaN      NaN    NaN       NaN   
          1775534231  0      NaN    NaN  NaN      NaN    NaN       NaN   

                          service area  
u         v           key               
135401287 8274769613  0       NaN  NaN  
          8274640348  0       NaN  NaN  
          8274769525  0       NaN  NaN  
135827942 11244868511 0       NaN  NaN  
          1775534231  0       NaN  NaN